In [1]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame

In [2]:
def remove_empty_space(df):
    for i in range(0,len(df)):
        df.code[i] = str(df.code[i]).replace(' ', '')
        df.code[i] = str(df.code[i]).replace('nan', '')
        if df.code[i] =='':
            df.code[i]=None
    #for i in range(0,len(df)):
     #   df.code[i] = str(df.code[i]).replace('', None)
    return df 


def split(df):
    code = df['code'].str.split(';', expand=True)
    code.columns = ['code{}_df'.format(x+1) for x in code.columns]
    code_col_name = code.columns
    df = pd.concat([df, code], axis=1)
    return df, code_col_name


# using lambda
# Empty String to None Conversion
def emptystr_to_None(df, code_col_name):
    conv = lambda i : i or None
    for j in code_col_name:
        df[j] = [conv(i) for i in df[j]]
    return df 


def pre_target_df(df,code_col_name):
    for i in code_col_name:
        df[i] = df['sql_id'].astype(str) + df[i]
    df = df.drop(columns=['code', 'coder','sql_id']) 
    return df 
  
    
def get_unique_sql_error(df,code_col_name):
    unique_sql_error =[]
    total_sql_error = []
    for i in code_col_name:
        total_sql_error= total_sql_error + list(df[i])
        unique_sql_error= unique_sql_error + list(df[i].unique())
    d = DataFrame(unique_sql_error,columns= ['sql_code'])
    unique = list(d.sql_code.unique())
    total_unique = len(unique)
    total_error = len(total_sql_error)
    #conv = lambda i : i or None
    #unique= [conv(i) for i in unique]
    return unique, total_unique, total_error

'''record_id'''
def stability_df(df, recordid_list, code_col_name, unique_sql_error ):
    d = DataFrame(0,columns= unique_sql_error, index = recordid_list)
    ls =[]
    l =[]
    for x in code_col_name:
        x = df[['record_id',x]].values.tolist() # batch1
        print(x)
        ls.append(x)
    print(ls)
    for colcode in ls: #colcode =[['record_id','code'],['record_id','code'], ...]
        for i in colcode: # i=['record_id','code']
            for j in range(0,len(d.index)):
                if i[0] ==d.index[j]:
                    d[i[1]][j]=d[i[1]][j]+1
                    l.append(i)
    return d, l

'''sql_id'''
def sqlstability_df(df, recordid_list, code_col_name, unique_sql_error ):
    d = DataFrame(0,columns= unique_sql_error, index = recordid_list)
    ls =[]
    l =[]
    for x in code_col_name:
        x = df[['sql_id',x]].values.tolist()
        ls.append(x)
    for colcode in ls: #colcode =[['sql_id','code'],['sql_id','code'], ...]
        for i in colcode: # i=['sql_id','code']
            for j in range(0,len(d.index)):
                if i[0] ==d.index[j]:
                    #print(d[i[1]][j])
                    d[i[1]][i[0]]+=1
                    l.append(i)
    return d, l


def total(matrix):
    total = 0
    for i in range(0,len(matrix.index)): 
        total = total+ sum(matrix.iloc[i])
    return total

---

In [40]:
#/kaggle/input/batch5annotationtask/batch5_annotationtask_sql_errortype.csv
#/kaggle/input/batch5annotationtask/batch5_annotationtask_sql_errorcategory.csv
#batch2
import os
for dirname, _, filenames in os.walk('/Users/tianyisun/Desktop/NLQbatches/batch6'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/tianyisun/Desktop/NLQbatches/batch6/RealAnalysisComplete2019.pdf
/Users/tianyisun/Desktop/NLQbatches/batch6/correct_batch6.csv
/Users/tianyisun/Desktop/NLQbatches/batch6/correct_batch6_3_.csv
/Users/tianyisun/Desktop/NLQbatches/batch6/correct_batch6_nz.csv
/Users/tianyisun/Desktop/NLQbatches/batch6/batch6_category_Fleiss_kappa.csv
/Users/tianyisun/Desktop/NLQbatches/batch6/.DS_Store
/Users/tianyisun/Desktop/NLQbatches/batch6/batch6_sql-type_Fleiss_kappa.csv
/Users/tianyisun/Desktop/NLQbatches/batch6/correct_batch6_anntask_sql_errorcategory.csv
/Users/tianyisun/Desktop/NLQbatches/batch6/correct_batch6_sql-category_Fleiss_kappa.csv
/Users/tianyisun/Desktop/NLQbatches/batch6/batch6_sql-category_Krippendroffis_alpha.csv
/Users/tianyisun/Desktop/NLQbatches/batch6/batch6_sql-type_Krippendroffis_alpha.csv
/Users/tianyisun/Desktop/NLQbatches/batch6/batch6_sql-category_agreement.csv
/Users/tianyisun/Desktop/NLQbatches/batch6/batch6_category_Krippendroffis_alpha.csv
/Users/tianyisun/Deskt

In [22]:
'''batch 5 error errortype'''
batch2 = pd.read_csv('/Users/tianyisun/Desktop/NLQbatches/batch4/correct_batch4_anntask_sql_type.csv')
#batch2 = pd.read_csv('/Users/tianyisun/Desktop/NLQbatches/batch7/correct_batch4567_anntask_sql_errorcategory.csv')
batch2 = batch2[['record_id','sql_id','code','username']]
batch2 = batch2.rename(columns ={'username':'coder'}) 
#batch2 = batch2.rename(columns ={'error_classification':'code'}) 
tianyi = batch2[batch2.coder =='tianyi'] # 56
zhengz = batch2[batch2.coder =='zhengzhang'] # 65
zhengn = batch2[batch2.coder =='zhengn'] # 66
toby = batch2[batch2.coder =='toby'] # 66
sqlid_list = list(tianyi.sql_id.unique())  # zhengn 45; zhengz 44; toby 46; tianyi 43

In [23]:
zhengn, zhengn_code_col_name = split(zhengn)
zhengn = emptystr_to_None(zhengn, zhengn_code_col_name)
zhengn = zhengn[['sql_id','code1_df']]
zhengn_unique_sql_error, total_unique, zhengn_total_sql_error = get_unique_sql_error(zhengn, zhengn_code_col_name)
zhengn_stability_df, mapping= sqlstability_df(zhengn, sqlid_list, zhengn_code_col_name, zhengn_unique_sql_error)
zhengn_stability_df_total = total(zhengn_stability_df)
print(zhengn_total_sql_error) # 552  89 batch3: 66  87
print(zhengn_stability_df_total) # 528 89 batch3: 63 84
zhengn_stability_df.head(3)

ls=[]
for i in range(0, len(zhengn_stability_df.columns)):
    ls.append(list(zhengn_stability_df.index[zhengn_stability_df.iloc[:,i] == 1]))
d_zhengn = DataFrame(ls, index = list(zhengn_stability_df.columns))  
#d_zhengn.T # 46

2507
2504


In [24]:
tianyi, tianyi_code_col_name = split(tianyi)
tianyi = emptystr_to_None(tianyi, tianyi_code_col_name)
tianyi = tianyi[['sql_id','code1_df']]
#tianyi = pre_target_df(tianyi, tianyi_code_col_name)
tianyi_unique_sql_error, total_unique, tianyi_total_sql_error = get_unique_sql_error(tianyi, tianyi_code_col_name)
tianyi_stability_df, mapping= sqlstability_df(tianyi, sqlid_list, tianyi_code_col_name, tianyi_unique_sql_error )
tianyi_stability_df_total = total(tianyi_stability_df)
print(tianyi_total_sql_error) # 724 94  batch3:56  81
print(tianyi_stability_df_total) # 724 84  batch3:56 81
tianyi_stability_df.head(3)

ls=[]
for i in range(0, len(tianyi_stability_df.columns)):
    ls.append(list(tianyi_stability_df.index[tianyi_stability_df.iloc[:,i] == 1]))
d_tianyi = DataFrame(ls, index = list(tianyi_stability_df.columns))  
#d_tianyi.T # 34 

2504
2504


In [61]:
toby, toby_code_col_name = split(toby)
toby = emptystr_to_None(toby, toby_code_col_name)
toby = toby[['sql_id','code1_df']]
toby_unique_sql_error, total_unique, toby_total_sql_error = get_unique_sql_error(toby, toby_code_col_name)
toby_stability_df, mapping= sqlstability_df(toby, sqlid_list, toby_code_col_name, toby_unique_sql_error )
toby_stability_df_total = total(toby_stability_df)
print(toby_total_sql_error) # 724 94  batch3:56  81
print(toby_stability_df_total) # 724 84  batch3:56 81
toby_stability_df.head(3)

ls=[]
for i in range(0, len(toby_stability_df.columns)):
    ls.append(list(toby_stability_df.index[toby_stability_df.iloc[:,i] == 1]))
d_toby = DataFrame(ls, index = list(toby_stability_df.columns))  
d_toby.T # 34 

KeyError: "['code1_df'] not in index"

In [9]:
zhengz, zhengz_code_col_name = split(zhengz)
zhengz = emptystr_to_None(zhengz, zhengz_code_col_name)
zhengz = zhengz[['sql_id','code1_df']]
zhengz_unique_sql_error, total_unique, zhengz_total_sql_error = get_unique_sql_error(zhengz, zhengz_code_col_name)
zhengz_stability_df, mapping= sqlstability_df(zhengz, sqlid_list, zhengz_code_col_name, zhengz_unique_sql_error )
zhengz_stability_df_total = total(zhengz_stability_df)
print(zhengz_total_sql_error) # 724 94  batch3:56  81
print(zhengz_stability_df_total) # 724 84  batch3:56 81
zhengz_stability_df.head(3)

ls=[]
for i in range(0, len(zhengz_stability_df.columns)):
    ls.append(list(zhengz_stability_df.index[zhengz_stability_df.iloc[:,i] == 1]))
d_zhengz = DataFrame(ls, index = list(zhengz_stability_df.columns))  
d_zhengz.T # 34 

KeyError: "['code1_df'] not in index"

----
----
section2

In [25]:
#result = pd.concat([d_tianyi.T, d_toby.T, d_zhengz.T, d_zhengn.T], ignore_index=True, sort=False)
result = pd.concat([d_tianyi.T, d_zhengn.T], ignore_index=True, sort=False)
'''batch1'''
#result.iloc[0:34] # d_tianyi  result[0:24]
#result.iloc[34: 78] # d_toby  result[24:48]
#result.iloc[78: 129] # d_zhengz  result[48:72]
#result.iloc[129:175] # d_zhengn  result[72:]
'''batch2'''                    '''category'''
#result.iloc[0:11]# d_tianyi    result[0:20]
#result.iloc[11:22] # d_toby    result[20:39]
#result.iloc[22:32] # d_zhengz  result[39:60]
#result.iloc[32:]# d_zhengn     result[60:]
'''batch3''' 
#result.iloc[0:7] # d_tianyi   [0:15]
#result.iloc[7:18] # d_toby    [15:33]
#result.iloc[18:28] # d_zhengz   [33:50]
#result.iloc[28:] # d_zhengn    [50:]
'''batch4''' 
#result.iloc[0:103] (category)  d_tianyi   [:58]  (type)
#result.iloc[103::]             d_zhengn   [58:]
'''batch5''' 
#result.iloc[0:197] (category)  d_tianyi  result.iloc[0:81]  (type)
#result.iloc[197::]             d_zhengn  result.iloc[81::]
'''batch6''' 
#result.iloc[0:171] (category)  d_tianyi  result.iloc[0:79]  (type)
#result.iloc[171::]             d_zhengn  result.iloc[79::]

'batch6'

In [26]:
result

,F1,F2,D1a,A1a,KA1b,CL1,BN1,AN2,CE2,A1f,...,O1,cL1,O3,BK1,P1,R1,A2c,A2b,CE1,NaN
0,1024.0,1024.0,1025.0,1025.0,1025.0,1026.0,689.0,1027.0,1029.0,1030.0,...,350.0,145.0,35.0,24.0,220.0,579.0,731.0,931.0,944.0,67.0
1,1033.0,674.0,595.0,1078.0,900.0,1028.0,715.0,1030.0,1036.0,327.0,...,138.0,NaN,265.0,103.0,381.0,NaN,747.0,NaN,NaN,62.0
2,674.0,105.0,385.0,989.0,1065.0,1036.0,35.0,840.0,1049.0,830.0,...,185.0,NaN,232.0,50.0,380.0,NaN,760.0,NaN,NaN,63.0
3,35.0,341.0,386.0,171.0,1092.0,1043.0,1088.0,830.0,897.0,284.0,...,570.0,NaN,301.0,374.0,201.0,NaN,925.0,NaN,NaN,64.0
4,1071.0,106.0,819.0,932.0,979.0,1046.0,748.0,828.0,900.0,971.0,...,917.0,NaN,228.0,377.0,356.0,NaN,16.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,NaN,NaN,NaN,NaN,NaN,322.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450,NaN,NaN,NaN,NaN,NaN,336.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
451,NaN,NaN,NaN,NaN,NaN,343.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
452,NaN,NaN,NaN,NaN,NaN,352.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
'''batch7'''
result.iloc[0:171] # [0:106] #(category)  d_tianyi  result.iloc[0:79]  (type)
result.iloc[212::] # [106::] #          d_zhengn  result.iloc[79::]

,F1,F2,D1a,A1a,KA1b,CL1,BN1,AN2,CE2,A1f,...,O1,cL1,O3,BK1,P1,R1,A2c,A2b,CE1,NaN
212,674.0,1024.0,595.0,1025.0,1025.0,1026.0,689.0,1027.0,1029.0,830.0,...,350.0,NaN,35.0,24.0,220.0,579.0,747.0,NaN,NaN,NaN
213,35.0,286.0,385.0,1078.0,900.0,689.0,696.0,1030.0,1034.0,284.0,...,138.0,NaN,265.0,50.0,381.0,NaN,760.0,NaN,NaN,NaN
214,1073.0,199.0,386.0,989.0,1065.0,1027.0,715.0,327.0,1035.0,229.0,...,185.0,NaN,232.0,377.0,1021.0,NaN,963.0,NaN,NaN,NaN
215,633.0,105.0,819.0,171.0,1092.0,1028.0,1088.0,840.0,1036.0,344.0,...,570.0,NaN,301.0,293.0,380.0,NaN,964.0,NaN,NaN,NaN
216,987.0,341.0,518.0,1.0,979.0,696.0,220.0,830.0,897.0,419.0,...,615.0,NaN,228.0,392.0,201.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,NaN,NaN,NaN,NaN,NaN,322.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450,NaN,NaN,NaN,NaN,NaN,336.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
451,NaN,NaN,NaN,NaN,NaN,343.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
452,NaN,NaN,NaN,NaN,NaN,352.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
'''batch 4- when user = 2 '''
tianyi_set = []
zhengn_set = []
#zhengz_set = []
#toby_set = []

intersection2_id = []
intersection2 =[]
union_id = []
union=[]
'''batch3'''
#result.iloc[0:171] (category)  d_tianyi  [0:20]  (type)
#result.iloc[63::]             d_zhengn 
for col in result.columns:
    tianyi_set.append(set([i for i in result.iloc[0:212][col] if str(i) != 'nan' and str(i)!='None']))
    zhengn_set.append(set([i for i in result.iloc[212::][col] if str(i) != 'nan' and str(i)!='None']))
    
    u= len([i for i in result[col].unique() if str(i) != 'nan' and str(i)!='None'])
    union.append(u)
    union_id.append(set([i for i in result[col].unique() if str(i) != 'nan' and str(i)!='None']))
    intersection2_id.append(set([i for i in result.iloc[0:212][col] if str(i) != 'nan' and str(i)!='None']) 
                            & set([i for i in result.iloc[212::][col] if str(i) != 'nan' and str(i)!='None']))
    intersection2.append(len(set([i for i in result.iloc[0:212][col] if str(i) != 'nan' and str(i)!='None']) & set([i for i in result.iloc[212::][col] if str(i) != 'nan' and str(i)!='None'])))
    
    

'''batch1'''
#result.iloc[0:34] # d_tianyi  result[0:24]
#result.iloc[34: 78] # d_toby  result[24:48]
#result.iloc[78: 129] # d_zhengz  result[48:72]
#result.iloc[129:175] # d_zhengn  result[72:] 
tianyi_set = []
toby_set = []
zhengz_set = []
zhengn_set = []

intersection4_id = []
intersection4 =[]
intersection3_id = []
intersection3 =[]
union_id = []
union=[]
for col in result.columns:
    tianyi_set.append(set([i for i in result.iloc[0:24][col] if str(i) != 'nan' and str(i)!='None']))
    toby_set.append(set([i for i in result.iloc[24:48][col] if str(i) != 'nan' and str(i)!='None']))
    zhengz_set.append(set([i for i in result.iloc[48:72][col] if str(i) != 'nan' and str(i)!='None']))
    zhengn_set.append(set([i for i in result.iloc[72:][col] if str(i) != 'nan' and str(i)!='None']))
    
    u= len([i for i in result[col].unique() if str(i) != 'nan' and str(i)!='None'])
    union.append(u)
    union_id.append(set([i for i in result[col].unique() if str(i) != 'nan' and str(i)!='None']))
    intersection4_id.append(set([i for i in result.iloc[0:24][col] if str(i) != 'nan' and str(i)!='None']) 
                            & set([i for i in result.iloc[24:48][col] if str(i) != 'nan' and str(i)!='None']) 
                            &set([i for i in result.iloc[48:72][col] if str(i) != 'nan' and str(i)!='None'])
                            & set([i for i in result.iloc[72:][col] if str(i) != 'nan' and str(i)!='None']))
    intersection4.append(len(set([i for i in result.iloc[0:24][col] if str(i) != 'nan' and str(i)!='None']) 
                             & set([i for i in result.iloc[24:48][col] if str(i) != 'nan' and str(i)!='None']) 
                             &set([i for i in result.iloc[48:72][col] if str(i) != 'nan' and str(i)!='None'])
                             & set([i for i in result.iloc[72:][col] if str(i) != 'nan' and str(i)!='None'])))
    
    A = set([i for i in result.iloc[0:24][col] if str(i) != 'nan' and str(i)!='None'])& set([i for i in result.iloc[24:48][col] if str(i) != 'nan' and str(i)!='None']) &set([i for i in result.iloc[48:72][col] if str(i) != 'nan' and str(i)!='None'])                         
    B = set([i for i in result.iloc[0:24][col] if str(i) != 'nan' and str(i)!='None'])& set([i for i in result.iloc[24:48][col] if str(i) != 'nan' and str(i)!='None'])& set([i for i in result.iloc[72:][col] if str(i) != 'nan' and str(i)!='None'])
    C = set([i for i in result.iloc[0:24][col] if str(i) != 'nan' and str(i)!='None']) &set([i for i in result.iloc[48:72][col] if str(i) != 'nan' and str(i)!='None'])& set([i for i in result.iloc[72:][col] if str(i) != 'nan' and str(i)!='None'])
    D = set([i for i in result.iloc[24:48][col] if str(i) != 'nan' and str(i)!='None']) &set([i for i in result.iloc[48:72][col] if str(i) != 'nan' and str(i)!='None'])& set([i for i in result.iloc[72:][col] if str(i) != 'nan' and str(i)!='None'])                       
    intersection3_id.append(A.union(B, C, D))
    intersection3.append(len(A.union(B, C, D)))    
    

---
---

'''batch 1-3 when user = 4 '''
df = DataFrame(columns= result.columns)
df.loc[len(df.index)] = toby_set
df = df.rename( index={len(df.index)-1: 'toby'})
df.loc[len(df.index)] = zhengz_set
df = df.rename( index={len(df.index)-1: 'zhengz'})
df.loc[len(df.index)] = zhengn_set
df = df.rename( index={len(df.index)-1: 'zhengn'})
df.loc[len(df.index)] = tianyi_set
df = df.rename( index={len(df.index)-1: 'tianyi'})

df.loc[len(df.index)] = intersection4_id
df = df.rename( index={len(df.index)-1: 'intersection4_id'})
df.loc[len(df.index)] = intersection3_id
df = df.rename( index={len(df.index)-1: 'intersection3_id'})
df.loc[len(df.index)] = union_id
df = df.rename( index={len(df.index)-1: 'union_id'})
df.loc[len(df.index)] = intersection4
df = df.rename( index={len(df.index)-1: 'intersection4'})
df.loc[len(df.index)] = intersection3
df = df.rename( index={len(df.index)-1: 'intersection3'})
df.loc[len(df.index)] = union
df = df.rename( index={len(df.index)-1: 'union'})

In [33]:
'''batch 4- when user = 2 '''
df = DataFrame(columns= result.columns)
df.loc[len(df.index)] = zhengn_set
df = df.rename( index={len(df.index)-1: 'zhengn'})
df.loc[len(df.index)] = tianyi_set
df = df.rename( index={len(df.index)-1: 'tianyi'})

df.loc[len(df.index)] = intersection2_id
df = df.rename( index={len(df.index)-1: 'intersection2_id'})
df.loc[len(df.index)] = union_id
df = df.rename( index={len(df.index)-1: 'union_id'})
df.loc[len(df.index)] = intersection2
df = df.rename( index={len(df.index)-1: 'intersection2'})
df.loc[len(df.index)] = union
df = df.rename( index={len(df.index)-1: 'union'})

inter4 =['']*len(df.columns)
inter3ormore =['']*len(df.columns)
for i in range(0,len(df.iloc[6,:])):
    if df.iloc[6,:][i]==4:
        inter4[i] = '*'
    if df.iloc[6,:][i]>=3:
        inter3ormore[i] = '*'
df.loc[len(df.index)] = inter4
df = df.rename( index={len(df.index)-1: 'inter==4'})
df.loc[len(df.index)] = inter3ormore
df = df.rename( index={len(df.index)-1: 'inter>=3'})

'''batch1-3 category stability'''
interdivunion=[]
for i in range(0,len(union)):
    if union[i] ==0:
        interdivunion.append('nan')
    else:
        interdivunion.append(intersection4[i]/union[i])
df.loc[len(df.index)] = interdivunion
df = df.rename( index={len(df.index)-1: 'inter4/union'})

inter3divunion=[]
for i in range(0,len(union)):
    if union[i] ==0:
        inter3divunion.append('nan')
    else:
        inter3divunion.append(intersection3[i]/union[i])
df.loc[len(df.index)] = inter3divunion
df = df.rename( index={len(df.index)-1: 'inter3/union'})
df

In [34]:
'''batch 4-5 when user = 2 '''
interdivunion=[]
for i in range(0,len(union)):
    if union[i] ==0:
        interdivunion.append('nan')
    else:
        interdivunion.append(intersection2[i]/union[i]) # change 'intersection4' -> 'intersection2'
df.loc[len(df.index)] = interdivunion
df = df.rename( index={len(df.index)-1: 'inter2/union'}) # 'inter4' -> 'inter2'

In [36]:
df.to_csv('batch7/correct_batch4567_type_stability.csv', index = True)
#df.to_csv('batch7/correct_batch4567_category_stability.csv', index = True)
df

,F1,F2,D1a,A1a,KA1b,CL1,BN1,AN2,CE2,A1f,...,O1,cL1,O3,BK1,P1,R1,A2c,A2b,CE1,NaN
zhengn,"{516.0, 140.0, 272.0, 17.0, 401.0, 402.0, 784....","{1024.0, 768.0, 769.0, 199.0, 647.0, 105.0, 81...","{518.0, 522.0, 523.0, 17.0, 25.0, 1050.0, 1052...","{1025.0, 1.0, 171.0, 111.0, 277.0, 1078.0, 989.0}","{1025.0, 900.0, 391.0, 775.0, 776.0, 777.0, 14...","{1026.0, 1027.0, 1028.0, 3.0, 515.0, 6.0, 9.0,...","{5.0, 522.0, 24.0, 560.0, 1088.0, 582.0, 583.0...","{1027.0, 388.0, 1030.0, 138.0, 268.0, 525.0, 6...","{1029.0, 5.0, 1034.0, 1035.0, 1036.0, 526.0, 5...","{350.0, 419.0, 229.0, 5.0, 779.0, 428.0, 429.0...",...,"{615.0, 138.0, 778.0, 185.0, 570.0, 350.0}",{},"{35.0, 228.0, 227.0, 232.0, 265.0, 301.0, 116....","{128.0, 774.0, 391.0, 392.0, 23.0, 280.0, 281....","{577.0, 578.0, 356.0, 772.0, 773.0, 357.0, 201...",{579.0},"{760.0, 747.0, 964.0, 963.0}",{},{},{}
tianyi,"{1024.0, 1.0, 516.0, 1033.0, 140.0, 272.0, 17....","{1024.0, 768.0, 769.0, 674.0, 807.0, 808.0, 81...","{1025.0, 518.0, 523.0, 17.0, 25.0, 1050.0, 105...","{1025.0, 963.0, 932.0, 933.0, 964.0, 171.0, 10...","{1025.0, 900.0, 775.0, 777.0, 140.0, 935.0, 10...","{1026.0, 3.0, 1028.0, 515.0, 6.0, 9.0, 10.0, 1...","{516.0, 5.0, 4.0, 533.0, 24.0, 35.0, 1088.0, 5...","{1027.0, 388.0, 1030.0, 138.0, 780.0, 653.0, 7...","{1029.0, 1036.0, 526.0, 529.0, 1049.0, 1067.0,...","{1030.0, 138.0, 778.0, 779.0, 653.0, 780.0, 78...",...,"{138.0, 917.0, 185.0, 570.0, 350.0}",{145.0},"{35.0, 228.0, 517.0, 227.0, 232.0, 265.0, 301....","{128.0, 774.0, 391.0, 392.0, 776.0, 917.0, 23....","{577.0, 578.0, 356.0, 772.0, 773.0, 357.0, 201...",{579.0},"{747.0, 111.0, 16.0, 760.0, 731.0, 925.0}",{931.0},{944.0},"{64.0, 67.0, 62.0, 63.0}"
intersection2_id,"{516.0, 140.0, 272.0, 17.0, 784.0, 663.0, 664....","{1024.0, 768.0, 769.0, 105.0, 814.0, 341.0, 62...","{518.0, 523.0, 17.0, 25.0, 1050.0, 1052.0, 105...","{1025.0, 171.0, 989.0, 1078.0}","{1025.0, 900.0, 775.0, 777.0, 140.0, 935.0, 10...","{1026.0, 515.0, 3.0, 1028.0, 6.0, 9.0, 10.0, 5...","{256.0, 257.0, 642.0, 643.0, 260.0, 5.0, 390.0...","{1027.0, 388.0, 1030.0, 138.0, 780.0, 653.0, 7...","{1029.0, 1036.0, 526.0, 1067.0, 1068.0, 47.0, ...","{419.0, 229.0, 779.0, 428.0, 429.0, 284.0, 830...",...,"{185.0, 138.0, 570.0, 350.0}",{},"{35.0, 228.0, 227.0, 232.0, 265.0, 301.0, 116....","{128.0, 774.0, 391.0, 392.0, 23.0, 280.0, 281....","{577.0, 578.0, 356.0, 773.0, 772.0, 357.0, 201...",{579.0},"{760.0, 747.0}",{},{},{}
union_id,"{1024.0, 1.0, 516.0, 1033.0, 140.0, 272.0, 17....","{1024.0, 768.0, 769.0, 647.0, 285.0, 286.0, 30...","{1025.0, 518.0, 522.0, 523.0, 17.0, 25.0, 1050...","{1025.0, 1.0, 963.0, 932.0, 933.0, 964.0, 171....","{1025.0, 900.0, 775.0, 391.0, 777.0, 776.0, 14...","{1026.0, 3.0, 1028.0, 515.0, 6.0, 1027.0, 9.0,...","{516.0, 5.0, 4.0, 522.0, 533.0, 24.0, 35.0, 56...","{1027.0, 388.0, 1030.0, 138.0, 780.0, 653.0, 7...","{1029.0, 5.0, 1019.0, 1034.0, 1035.0, 1036.0, ...","{5.0, 1030.0, 138.0, 778.0, 779.0, 653.0, 780....",...,"{615.0, 138.0, 778.0, 917.0, 185.0, 570.0, 350.0}",{145.0},"{35.0, 228.0, 517.0, 227.0, 232.0, 265.0, 301....","{128.0, 774.0, 391.0, 392.0, 776.0, 917.0, 23....","{577.0, 578.0, 356.0, 772.0, 773.0, 357.0, 201...",{579.0},"{963.0, 964.0, 747.0, 111.0, 16.0, 760.0, 731....",{931.0},{944.0},"{64.0, 67.0, 62.0, 63.0}"
intersection2,25,7,114,4,21,167,72,38,150,8,...,4,0,8,31,12,1,2,0,0,0
union,54,25,141,11,31,259,114,54,223,25,...,7,1,9,47,13,1,8,1,1,4
inter2/union,0.462963,0.28,0.808511,0.363636,0.677419,0.644788,0.631579,0.703704,0.672646,0.32,...,0.571429,0.0,0.888889,0.659574,0.923077,1.0,0.25,0.0,0.0,0.0


for i in range(0,len(tianyi)):
    if tianyi.code1_tianyi[i] ==str(''):
        tianyi.code1_tianyi[i] == tianyi.code3_tianyi[1]
    if tianyi.code2_tianyi[i] ==str(''):
        count+=1
        tianyi.code2_tianyi[i] == tianyi.code3_tianyi[1]
    if tianyi.code3_tianyi[i] ==str(''):
        count+=1
        tianyi.code3_tianyi[i] == tianyi.code3_tianyi[1]
    if tianyi.code4_tianyi[i] ==str(''):
        count+=1
        tianyi.code4_tianyi[i] == tianyi.code3_tianyi[1]
list(tianyi.code2_tianyi)
print(count)
print(strcount)

for i in tianyi_code_col_name:      
    tianyi[i] = tianyi["sql_id"].astype(str) + tianyi[i]
tianyi = tianyi.drop(columns=['code', 'coder','sql_id'])
ls = list(tianyi.code1_df.unique()) + list(tianyi.code2_df.unique()) +list(tianyi.code3_df.unique()) +list(tianyi.code4_df.unique())
l = DataFrame(ls,columns= ['sql_code'])
len(l.sql_code.unique())

---
---
---

## next step:
* compare each column of identical sql_error 
* if total is 2 then count+=1 
* count = 3 means overlap is 3 
* count = 4 means overlap is 4 